In [2]:
import pandas as pd
import seaborn as sns
import re
import matplotlib

In [3]:
def prepare_data(path):

    df = pd.read_csv(path, index_col=0)
    df[['mileage', 'engine_capacity', 'transmission', 'engine_power', 'body_type', 'wheel', 'fuel_type', 'model', 'model_2', 'year']] = None

    for i, row in df.loc[:, ['name', 'params', 'region']].iterrows():

        try:
            #Предобработка params
            mileage = re.search(r'([\d ]+) км,', row['params'], re.IGNORECASE)
            if mileage == None:
                mileage = 0
            else:
                df.loc[i, 'mileage'] = mileage.group(1).replace(' ', '')

            engine_capacity = re.search(r'([\d\.]+) [A-Z]+', row['params'])

            if engine_capacity != None:
                df.loc[i, 'engine_capacity'] = engine_capacity.group(1)

            df.loc[i, 'transmission'] = re.search(r'MT|CVT|AMT|AT', row['params'])[0]
            engine_power = re.search(r'\((\d+) л\.с\.\),', row['params'])

            if engine_power != None:
                df.loc[i, 'engine_power'] = engine_power.group(1)


            df.loc[i, 'body_type'] = re.search(r'([\w-]+), (полный|задний|передний)', row['params'], re.IGNORECASE).group(1)
            df.loc[i, 'wheel'] = re.search(r'полный|задний|передний', row['params'], re.IGNORECASE)[0]
            df.loc[i, 'fuel_type'] = re.search(r'бензин|дизель|электро|гибрид|газ', row['params'], re.IGNORECASE)[0]


            #предобработка name
            df.loc[i, 'model'] = re.search(r'[\w-]+ ', row['name'], re.IGNORECASE)[0]
            model_2 = re.search(r'[\w-]+ ([^,]*?) [\d\. ]*(AT|MT|AMT|CVT)?,', row['name'], re.IGNORECASE)

            if model_2 != None:
                df.loc[i, 'model_2'] = model_2.group(1)
            else:
                df.loc[i, 'model_2'] = re.search(r'[\w-]+ ([^,]*?),', row['name'], re.IGNORECASE).group(1)

            df.loc[i, 'year'] = re.search(r', ?(\d{4})', row['name']).group(1)

            #предобработка region
            df.loc[i, 'region'] = re.search(r'([^,]+),?', row['region'], re.IGNORECASE).group(1)

        except Exception as ex:
            print(row)
            print(ex)

    return df

In [ ]:
df = pd.DataFrame()
a = [0, 1, 3, 6, 7, 8, 9, 12, 13, 14, 15, 16, 18, 19, 22, 23, 25]

for i in a:
    #if i == 0:
    #    res = prepare_data(f'./datasets/data.csv')
    #else:
    res = prepare_data(f'./datasets/data{i}.csv')
    
    df = pd.concat([df, res])
    print(i)


In [5]:
df.to_csv('./dataset2.csv')

In [12]:
a1 = pd.read_csv('./dataset.csv', index_col=0)
a2 = pd.read_csv('./dataset2.csv', index_col=0)

a = pd.concat([a1, a2])
a.to_csv('./dataset_all.csv')